In [39]:
# !pip install transformers
# !pip install -U "huggingface_hub[cli]" 
# import os
# !huggingface-cli login --token os.gentenv('HUGGINGFACE_TOKEN') --add-to-git-credential
# !pip install bitsandbytes
# !pip install accelerate>=0.26.0
# !pip install protobuf
# !pip install sentencepiece 
# !pip install llama-index

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

C:\Users\Eric\.conda\envs\python310gpu\lib\site-packages\torch\_subclasses\functional_tensor.py:275: UserWarning: Failed to initialize NumPy: DLL load failed while importing _multiarray_umath: The specified module could not be found. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))
C:\Users\Eric\.conda\envs\python310gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
torch.cuda.is_available()
print(torch.cuda.device_count())
print(torch.cuda.current_device())
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
print(available_gpus)

In [2]:
for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

NVIDIA GeForce RTX 2070 with Max-Q Design


In [3]:
model_name="mistralai/Mistral-7B-Instruct-v0.3"

In [4]:
bnb_config=BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_compute_dtype="float16")
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config = bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
# If loading from disk
# model_name_path=r"E:\AI_models\Mistral-7B-Instruct-v0.3"
# tokenizer = AutoTokenizer.from_pretrained(model_name_path, local_files_only=True)
# model = AutoModelForCausalLM.from_pretrained(model_name_path, local_files_only=True)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 3/3 [00:28<00:00,  9.55s/it]


In [6]:
model.save_pretrained(r"E:\AI_models\Mistral-7B-Instruct-v0.3")
tokenizer.save_pretrained(r"E:\AI_models\Mistral-7B-Instruct-v0.3")

('E:\\AI_models\\Mistral-7B-Instruct-v0.3\\tokenizer_config.json',
 'E:\\AI_models\\Mistral-7B-Instruct-v0.3\\special_tokens_map.json',
 'E:\\AI_models\\Mistral-7B-Instruct-v0.3\\tokenizer.model',
 'E:\\AI_models\\Mistral-7B-Instruct-v0.3\\added_tokens.json',
 'E:\\AI_models\\Mistral-7B-Instruct-v0.3\\tokenizer.json')

In [7]:
device = torch.device('cuda')

In [8]:
model.to(device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

In [9]:
prompt = (
    f"Create a social media post about the latest AI news:\n"
    f"Title: Latest AI News\n"
    f"Summary: OpenAI just released GPT 4.5\n"
    f"Read more: https://www.copewithtech.com\n"
    f"Post: "
)

In [10]:
messages = [
    {"role":"user","content":prompt},
]

In [12]:
inputs = tokenizer(prompt, return_tensors="pt")

In [15]:
inputs = inputs.to(device)
outputs = model.generate(
    inputs.input_ids,
    max_length=512,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    early_stopping=True
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [16]:
post = tokenizer.decode(outputs[0], skip_special_tokens=True)
post.split("Post:")[-1].strip()

"🤖𝗦\U0001e983𑁍𐌄𒀀𓃠𖠊\U00015314\U000192c4 🚀\n\nWe're excited to announce that Open AI has just unveiled GTP 5.0! This latest advancement in AI technology is set to revolutionize the way we interact with machines. 🌍\U0001410d\n𘐀\nStay tuned for more updates and insights on this groundbreaking development. Don't miss out! 👀\n👉 https:\\/\\/www\\. copewith\\ tech\\ .com 👉\n#AI #GPT45 #OpenAI\nLet's explore the future together! #TechTrends #Innovation #AI4All"

In [37]:
prompt = ""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
while prompt!="q":
    user_input=input("Enter your prompt: ")
    if user_input=="q":
        break

    prompt = (
        f"User: {user_input}\n"
        "AI Assistant (If you are unsure or the information is not in your training data, respond with I do not know):\n"
    )
    
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    inputs = inputs.to(device)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    outputs = model.generate(
        inputs.input_ids,
        max_length=1024,
        max_new_tokens=300,
        temperature=0.3,
        top_p=0.8,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        attention_mask=attention_mask
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_response = generated_text.split("AI Assistant (If you are unsure or the information is not in your training data, respond with I do not know):")[-1].strip()

    if "I do not know" in generated_response or "I am not sure" in generated_response:
        final_response=generated_response
    elif len(generated_response.split()) < 5: 
        final_response="I am sorry but I do not know."
    else:
        final_response=generated_response

    print(final_response)
        

Enter your prompt:  It can be one of those typically lukewarm model responses with lots of caveats, long-winded explanations, or half-hallucinated knowledge from the day before yesterday.  Or the first section of a best-selling novel, a proposal for a perfect brand name, the most powerful and elegant Python function for a tricky problem.  Or even more, more and more of that: When you are building an application, the model can reliably and accurately answer millions of questions from your customers, fully process insurance claims or stubbornly rummage every day through the freshly filed patent filings in search of conflicts with older ones.


Both `max_new_tokens` (=300) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


I'm sorry, but I don't have enough context or information to provide a detailed response. If you could provide more details or clarify your question, I would be happy to help!


Enter your prompt:  The quick brown fox jumps over what?


Both `max_new_tokens` (=300) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Here's a simple Python script for computing the first n numbers of the fibonnaci sequence:
```python
def fib(n): # function to compute the nth number in the sequence
    if n <= 1: # base case
        return n
   else:      # recursive case, compute n-1 and n+1 then add them together
       return fib((n - 2)) + fib ((n- 3))
# function main to test the above function
if __name__ == "__main__":
     n = int(input("Enter the number of terms:")) # get input from user
      print("The first {} numbers in FIBONACCI sequence are:".format(str( n)))
  # print the result
 for i in range(0, n):  print(fib(i), end=" ")
 print()
 ```
 You can run this script by copying it into a file named `f_sequence.py`, then running it using the command `python f_sequenc.p`. The script will prompt you to enter the desired number `n` of Fubonaccci sequence terms, then output the requested sequence numbers.


Enter your prompt:  Create a social media post about the article that can be found at https://medium.com/the-generator/the-perfect-prompt-prompt-engineering-cheat-sheet-d0b9c62a2bba


Both `max_new_tokens` (=300) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🔬𝗦𞋄𖠊 𓃠𒀀𕌆𑁍𐅡𔵈𘂚𛿏𙇎𜢉𠮷𣺱𤭩𥧲𦯳𨹪𩫼𧰸𡴾𢻄 ❄️𗿔𚂽𫶠 🎉
🚀🚀𬠂𭍄🚀
The Perfect Prompt: A Comprehensive Guide to Prompts Engineering 🚀✨🚀 - Your ultimate cheat sheet for crafting AI-friendly prompts! 🌍𪀇𯺎🌍
👉 [https://bit.ly/3z7JjKp](https:/bitly.3/z/7/J/j/K/p)
#PromptEngineering #AI #ArtificialIntelligence #CheatSheet #WritingPrompts #Creativity #Innovation #TechLife #MachineLearning #NLP #LanguageModel #TextGeneration #DeepLearing #Coding #Programming #DataScience #Technology #DigitalTransformation #FutureOfWork #Robotics #ChatGPT #Bard #E


Enter your prompt:  q


In [ ]:
post = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
post.split("Post:")[-1].strip()

In [ ]:
model = None
inputs = None
outputs = None
tokenizer = None

model2 = None
tokenizer2 = None

In [ ]:
with torch.no_grad():
  torch.cuda.empty_cache()